# CS 4501 Machine Learning Project
**Team Members:** Leo Wang (yw7uc), Alicia Wu (yw7vv), Simon Zhu (mz4cr)
1. [Data Acquiring](#data_acquiring)
2. [Data Pre-processing](#data_pre-processing)
3. [Data Splitting](#data_splitting)
4. [Data Discovery](#data_discovery)
5. [Data Cleaning and Feature Scaling](#cleaning_and_scaling)
6. [Data Training](#data_training)

---
## 1. Data Acquiring <a name="data_acquiring"></a>

In [0]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

from mlxtend.classifier import SoftmaxRegression
from pandas.plotting import scatter_matrix # optional
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer, LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.svm import LinearSVC, SVC

np.random.seed(42)

import warnings
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning, UndefinedMetricWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UndefinedMetricWarning)
warnings.filterwarnings(action = 'ignore', category = DataConversionWarning)
warnings.filterwarnings(action = 'ignore', category = ConvergenceWarning)

flight = pd.read_csv('flights_and_weather_drop_delay_missing.csv', low_memory=False)
flight.head()

,FlightDate,Reporting_Airline,CRSDepTime,DepDelayMinutes,DepTimeBlk,Cancelled,Distance,PRCP,SNOW,TMIN
0,2015-01-01,AA,625,0.0,0600-0659,0.0,1172.0,0.00,0.0,15.0
1,2015-01-02,AA,625,0.0,0600-0659,0.0,1172.0,0.00,0.0,16.0
2,2015-01-03,AA,625,0.0,0600-0659,0.0,1172.0,0.03,0.0,23.0
3,2015-01-04,AA,625,0.0,0600-0659,0.0,1172.0,1.13,0.0,32.0
4,2015-01-05,AA,625,0.0,0600-0659,0.0,1172.0,0.10,0.0,32.0


---
## 2. Data Pre-processing <a name="data_pre-processing"></a>
1. [Modify some features from string class to numeric class](#str_to_num)
2. [Class to add delay category](#delay_cat_class)
3. [Class to add frozen category](#frozen_cat_class)
4. [Pipeline to add delay and frozen categories](#add_pipeline)
5. [Drop unnecessary features](#drop_unnecessary_features)
6. [Encode categorical features](#encode_categorical_features)

### 2.1. Modify some features from string class to numeric class <a name="str_to_num"></a>

In [0]:
cols = ['PRCP','SNOW']
flight[cols] = flight[cols].apply(pd.to_numeric, errors='coerce')

### 2.2. Class to add delay category <a name="delay_cat_class"></a>

In [0]:
class AddDelayCat(BaseEstimator, TransformerMixin):
    def __init__(self, add_delayCat=True): # no *args or **kargs
        self.add_delayCat = add_delayCat
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        row_num, col_num = X.shape[0], X.shape[1]
        delayTime_idx = 'DepDelayMinutes'
        cancelled_idx = 'Cancelled'
        X.insert(col_num, "DelayCategory", 0)
        delayCat_idx = "DelayCategory"

        for i in range(0, row_num):
            if X.loc[i, cancelled_idx] == 1.0: X.loc[i, delayCat_idx] = "Heavy Delay"
            elif X.loc[i, delayTime_idx] <= 30.0: X.loc[i, delayCat_idx] = "No Delay"
            elif 30.0 < X.loc[i, delayTime_idx] <= 120.0: X.loc[i, delayCat_idx] = "Slight Delay"
            else: X.loc[i, delayCat_idx] = "Heavy Delay"

        return X

### 2.3. Class to add frozen category <a name="frozen_cat_class"></a>

In [0]:
class AddFrozenCat(BaseEstimator, TransformerMixin):
    def __init__(self, add_frozen_cat=True): # no *args or **kargs
        self.add_frozen_cat = add_frozen_cat
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        row_num, col_num = X.shape[0], X.shape[1]
        PRCP_idx = 'PRCP'
        SNOW_idx = 'SNOW'
        TMIN_idx = 'TMIN'
        X.insert(col_num, "Frozen", 0)
        frozen_cat_idx = 'Frozen'
        
        for i in range(0, row_num):
            if (X.loc[i, PRCP_idx] > 0 or X.loc[i, SNOW_idx] > 0) and X.loc[i, TMIN_idx] <= 38: X.loc[i, frozen_cat_idx] = 1
            else: X.loc[i, TMIN_idx] = 0

        return X

### 2.4. Pipeline to add delay and frozen categories <a name="add_pipeline"></a>

In [0]:
add_pipeline = Pipeline([
    ('frozen', AddFrozenCat()),
    ('delay', AddDelayCat()),
])

flight_prepared = add_pipeline.fit_transform(flight)
flight_prepared.head()

,FlightDate,Reporting_Airline,CRSDepTime,DepDelayMinutes,DepTimeBlk,Cancelled,Distance,PRCP,SNOW,TMIN,Frozen,DelayCategory
0,2015-01-01,AA,625,0.0,0600-0659,0.0,1172.0,0.00,0.0,0.0,0,No Delay
1,2015-01-02,AA,625,0.0,0600-0659,0.0,1172.0,0.00,0.0,0.0,0,No Delay
2,2015-01-03,AA,625,0.0,0600-0659,0.0,1172.0,0.03,0.0,23.0,1,No Delay
3,2015-01-04,AA,625,0.0,0600-0659,0.0,1172.0,1.13,0.0,32.0,1,No Delay
4,2015-01-05,AA,625,0.0,0600-0659,0.0,1172.0,0.10,0.0,32.0,1,No Delay


### 2.5. Drop unnecessary features <a name="drop_unnecessary_features"></a>

In [0]:
attributes = ['Reporting_Airline', 'DepTimeBlk', 'Distance', 'PRCP', 'SNOW', 'TMIN', 'Frozen', 'DelayCategory']
flight_prepared = flight_prepared[attributes]
flight_prepared.head()

,Reporting_Airline,DepTimeBlk,Distance,PRCP,SNOW,TMIN,Frozen,DelayCategory
0,AA,0600-0659,1172.0,0.00,0.0,0.0,0,No Delay
1,AA,0600-0659,1172.0,0.00,0.0,0.0,0,No Delay
2,AA,0600-0659,1172.0,0.03,0.0,23.0,1,No Delay
3,AA,0600-0659,1172.0,1.13,0.0,32.0,1,No Delay
4,AA,0600-0659,1172.0,0.10,0.0,32.0,1,No Delay


### 2.6. Encode categorical features <a name="encode_categorical_features"></a>
* Encode *Reporting_Airline* and *DepTimeBlk* in advance to avoid mismatch in train and test set after using OneHotEncoder

In [0]:
airline_encoded = pd.get_dummies(flight_prepared['Reporting_Airline'])
dep_time_encoded = pd.get_dummies(flight_prepared['DepTimeBlk'])
flight_encoded = pd.concat([flight_prepared, airline_encoded, dep_time_encoded], axis=1)
# flight_encoded = pd.concat([flight_prepared, airline_encoded], axis=1)
# flight_encoded = flight_prepared
flight_encoded = flight_encoded.drop('Reporting_Airline', axis=1)
flight_encoded = flight_encoded.drop('DepTimeBlk', axis=1)
flight_encoded.head()

,Distance,PRCP,SNOW,TMIN,Frozen,DelayCategory,9E,AA,AS,B6,...,1400-1459,1500-1559,1600-1659,1700-1759,1800-1859,1900-1959,2000-2059,2100-2159,2200-2259,2300-2359
0,1172.0,0.00,0.0,0.0,0,No Delay,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1172.0,0.00,0.0,0.0,0,No Delay,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1172.0,0.03,0.0,23.0,1,No Delay,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1172.0,1.13,0.0,32.0,1,No Delay,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1172.0,0.10,0.0,32.0,1,No Delay,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
flight_encoded

,Distance,PRCP,SNOW,TMIN,Frozen,DelayCategory,9E,AA,AS,B6,...,1400-1459,1500-1559,1600-1659,1700-1759,1800-1859,1900-1959,2000-2059,2100-2159,2200-2259,2300-2359
0,1172.0,0.00,0.0,0.0,0,No Delay,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1172.0,0.00,0.0,0.0,0,No Delay,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1172.0,0.03,0.0,23.0,1,No Delay,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1172.0,1.13,0.0,32.0,1,No Delay,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1172.0,0.10,0.0,32.0,1,No Delay,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1172.0,0.11,1.1,19.0,1,No Delay,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1172.0,0.18,3.1,11.0,1,No Delay,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1172.0,0.00,0.0,0.0,0,No Delay,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1172.0,0.00,0.0,0.0,0,No Delay,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1172.0,0.16,0.0,8.0,1,No Delay,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
# balanced_train_set = pd.DataFrame(columns=list(flight_encoded))
# no_delay_count = 3000
# slight_delay_count = 3000
# heavy_delay_count = 3000

# for i in range(96397):
#     if flight_encoded.iloc[i, 5] == 'No Delay' and no_delay_count > 0:
#         balanced_train_set.loc[i] = flight_encoded.iloc[i]
#         no_delay_count -= 1
#     elif flight_encoded.iloc[i, 5] == 'Slight Delay' and slight_delay_count > 0:
#         balanced_train_set.loc[i] = flight_encoded.iloc[i]
#         slight_delay_count -= 1
#     elif flight_encoded.iloc[i, 5] == 'Heavy Delay' and heavy_delay_count > 0:
#         balanced_train_set.loc[i] = flight_encoded.iloc[i]
#         heavy_delay_count -= 1
        
# balanced_train_set

---
## 3. Data Splitting <a name="data_splitting"></a>

In [0]:
train_set, test_set = train_test_split(flight_encoded, test_size=0.2, random_state=42)
# train_set, test_set = train_test_split(balanced_train_set, test_size=0.2, random_state=42)

X_train = train_set.drop('DelayCategory', axis=1)
# X_train = train_set[['Distance', 'PRCP', 'SNOW', 'TMIN', 'Frozen']]
y_train = train_set[['DelayCategory']]
X_test = test_set.drop('DelayCategory', axis=1)
# X_test = test_set[['Distance', 'PRCP', 'SNOW', 'TMIN', 'Frozen']]
y_test = test_set[['DelayCategory']]

---
## 4. Data Discovery <a name="data_discovery"></a>

In [0]:
flight_prepared.corr()

,Distance,PRCP,SNOW,TMIN,Frozen
Distance,1.000000,-0.013017,-0.012905,0.011346,0.009003
PRCP,-0.013017,1.000000,0.127289,0.159678,0.139696
SNOW,-0.012905,0.127289,1.000000,0.258823,0.320052
TMIN,0.011346,0.159678,0.258823,1.000000,0.961899
Frozen,0.009003,0.139696,0.320052,0.961899,1.000000


---
## 5. Data Cleaning and Feature Scaling <a name="cleaning_and_scaling"></a>
1. [Detect missing value](#detect_missing_value)
2. [Scale and transform X](#scale_and_transform_X)

### 5.1. Detect missing value <a name="detect_missing_value"></a>

In [0]:
sample_incomplete_rows = flight_encoded[flight_encoded.isnull().any(axis = 1)]
sample_incomplete_rows.head()

,Distance,PRCP,SNOW,TMIN,Frozen,DelayCategory,9E,AA,AS,B6,...,1400-1459,1500-1559,1600-1659,1700-1759,1800-1859,1900-1959,2000-2059,2100-2159,2200-2259,2300-2359
9426,288.0,NaN,0.0,0.0,0,No Delay,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9428,908.0,NaN,0.0,0.0,0,No Delay,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9511,1452.0,NaN,0.0,0.0,0,No Delay,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9512,2419.0,NaN,0.0,0.0,0,No Delay,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
9513,2419.0,NaN,0.0,0.0,0,No Delay,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


### 5.2. Scale and transform X <a name="scale_and_transform_X"></a>

In [0]:
num_attributes = ['Distance', 'PRCP', 'SNOW', 'TMIN']

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler())
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attributes)
], remainder='passthrough')

X_train_prepared = full_pipeline.fit_transform(X_train)
X_test_prepared = full_pipeline.fit_transform(X_test)

---
## 6. Data Training <a name="data_training"></a>
1. [Logistic Regression](#logistic_regression)
2. [Linear SVM](#linear_svm)
3. [Polynomial Kernel SVM](#poly_kernel_svm)
4. [Gaussian RBF Kernel SVM](#rbf_kernel_svm)
5. [Random Forest](#random_forest)
5. [K Neighbors Classifier](#k_neighbors_classifier)

### 6.1. Softmax Logistic Regression <a name="logistic_regression"></a>

In [0]:
softmax_reg = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=0.01)
print("C = 0.01")
softmax_reg.fit(X_train_prepared, y_train)
y_pred = softmax_reg.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

C = 0.01
Accuracy =  88.37136929460581 %
Precision =  [88.37136929  0.          0.        ] %
Recall =  [100.   0.   0.] %
F1 score =  [93.82675257  0.          0.        ] %
[[17038     0     0]
 [ 1624     0     0]
 [  618     0     0]]


In [0]:
softmax_reg = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=0.1)
print("C = 0.1")
softmax_reg.fit(X_train_prepared, y_train)
y_pred = softmax_reg.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

C = 0.1
Accuracy =  88.37136929460581 %
Precision =  [88.37136929  0.          0.        ] %
Recall =  [100.   0.   0.] %
F1 score =  [93.82675257  0.          0.        ] %
[[17038     0     0]
 [ 1624     0     0]
 [  618     0     0]]


In [0]:
softmax_reg = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=1)
print("C = 1")
softmax_reg.fit(X_train_prepared, y_train)
y_pred = softmax_reg.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

C = 1
Accuracy =  88.37136929460581 %
Precision =  [88.37136929  0.          0.        ] %
Recall =  [100.   0.   0.] %
F1 score =  [93.82675257  0.          0.        ] %
[[17038     0     0]
 [ 1624     0     0]
 [  618     0     0]]


In [0]:
softmax_reg = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=10)
print("C = 10")
softmax_reg.fit(X_train_prepared, y_train)
y_pred = softmax_reg.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

C = 10
Accuracy =  88.37136929460581 %
Precision =  [88.37136929  0.          0.        ] %
Recall =  [100.   0.   0.] %
F1 score =  [93.82675257  0.          0.        ] %
[[17038     0     0]
 [ 1624     0     0]
 [  618     0     0]]


In [0]:
softmax_reg = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=100)
print("C = 100")
softmax_reg.fit(X_train_prepared, y_train)
y_pred = softmax_reg.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

C = 100
Accuracy =  88.37136929460581 %
Precision =  [88.37136929  0.          0.        ] %
Recall =  [100.   0.   0.] %
F1 score =  [93.82675257  0.          0.        ] %
[[17038     0     0]
 [ 1624     0     0]
 [  618     0     0]]


### 6.2. Linear SVM <a name="linear_svm"></a>

In [0]:
svm_clf = LinearSVC(C=0.01, loss="hinge", random_state=42)
print("C = 0.01")
svm_clf.fit(X_train_prepared, y_train)
y_pred = svm_clf.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

C = 0.01
Accuracy =  88.37136929460581 %
Precision =  [88.37136929  0.          0.        ] %
Recall =  [100.   0.   0.] %
F1 score =  [93.82675257  0.          0.        ] %
[[17038     0     0]
 [ 1624     0     0]
 [  618     0     0]]


In [0]:
svm_clf = LinearSVC(C=0.1, loss="hinge", random_state=42)
print("C = 0.1")
svm_clf.fit(X_train_prepared, y_train)
y_pred = svm_clf.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

C = 0.1
Accuracy =  88.37136929460581 %
Precision =  [88.37136929  0.          0.        ] %
Recall =  [100.   0.   0.] %
F1 score =  [93.82675257  0.          0.        ] %
[[17038     0     0]
 [ 1624     0     0]
 [  618     0     0]]


In [0]:
svm_clf = LinearSVC(C=1, loss="hinge", random_state=42)
print("C = 1")
svm_clf.fit(X_train_prepared, y_train)
y_pred = svm_clf.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

C = 1
Accuracy =  88.37136929460581 %
Precision =  [88.37136929  0.          0.        ] %
Recall =  [100.   0.   0.] %
F1 score =  [93.82675257  0.          0.        ] %
[[17038     0     0]
 [ 1624     0     0]
 [  618     0     0]]


### 6.3. Polynomial Kernal SVM <a name="poly_kernel_svm"></a>

In [0]:
# poly_kernel_svm_clf = SVC(kernel='poly', degree=2, coef0=1, C=1)
# print("Degree = 2, C = 1")
# poly_kernel_svm_clf.fit(X_train_prepared, y_train)
# y_pred = poly_kernel_svm_clf.predict(X_test_prepared)
# print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
# print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
# print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
# print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
# print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

### 6.4. Gaussian RBF Kernal SVM <a name="rbf_kernel_svm"></a>

In [0]:
# rbf_kernel_svm_clf = SVC(kernel='rbf', gamma=5, C=0.001)
# print("Gamma = 5, C = 0.001")
# rbf_kernel_svm_clf.fit(X_train_prepared, y_train)
# y_pred = rbf_kernel_svm_clf.predict(X_test_prepared)
# print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
# print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
# print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
# print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
# print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

### 6.5. Random Forest <a name="random_forest"></a>

In [0]:
rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=16, n_jobs=-1)
print("n_estimators = 100, max_leaf_nodes = 16")
rnd_clf.fit(X_train_prepared, y_train)
y_pred = rnd_clf.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

n_estimators = 100, max_leaf_nodes = 16
Accuracy =  88.37136929460581 %
Precision =  [88.37136929  0.          0.        ] %
Recall =  [100.   0.   0.] %
F1 score =  [93.82675257  0.          0.        ] %
[[17038     0     0]
 [ 1624     0     0]
 [  618     0     0]]


In [0]:
rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=64, n_jobs=-1)
print("n_estimators = 100, max_leaf_nodes = 64")
rnd_clf.fit(X_train_prepared, y_train)
y_pred = rnd_clf.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

n_estimators = 100, max_leaf_nodes = 64
Accuracy =  88.37136929460581 %
Precision =  [88.37136929  0.          0.        ] %
Recall =  [100.   0.   0.] %
F1 score =  [93.82675257  0.          0.        ] %
[[17038     0     0]
 [ 1624     0     0]
 [  618     0     0]]


In [0]:
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=8, n_jobs=-1)
print("n_estimators = 500, max_leaf_nodes = 8")
rnd_clf.fit(X_train_prepared, y_train)
y_pred = rnd_clf.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

n_estimators = 500, max_leaf_nodes = 8
Accuracy =  88.37136929460581 %
Precision =  [88.37136929  0.          0.        ] %
Recall =  [100.   0.   0.] %
F1 score =  [93.82675257  0.          0.        ] %
[[17038     0     0]
 [ 1624     0     0]
 [  618     0     0]]


In [0]:
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
print("n_estimators = 500, max_leaf_nodes = 16")
rnd_clf.fit(X_train_prepared, y_train)
y_pred = rnd_clf.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

n_estimators = 500, max_leaf_nodes = 16
Accuracy =  88.37136929460581 %
Precision =  [88.37136929  0.          0.        ] %
Recall =  [100.   0.   0.] %
F1 score =  [93.82675257  0.          0.        ] %
[[17038     0     0]
 [ 1624     0     0]
 [  618     0     0]]


In [0]:
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=64, n_jobs=-1)
print("n_estimators = 500, max_leaf_nodes = 64")
rnd_clf.fit(X_train_prepared, y_train)
y_pred = rnd_clf.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

n_estimators = 500, max_leaf_nodes = 64
Accuracy =  88.37136929460581 %
Precision =  [88.37136929  0.          0.        ] %
Recall =  [100.   0.   0.] %
F1 score =  [93.82675257  0.          0.        ] %
[[17038     0     0]
 [ 1624     0     0]
 [  618     0     0]]


In [0]:
rnd_clf = RandomForestClassifier(n_estimators=1000, max_leaf_nodes=16, n_jobs=-1)
print("n_estimators = 1000, max_leaf_nodes = 16")
rnd_clf.fit(X_train_prepared, y_train)
y_pred = rnd_clf.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

n_estimators = 1000, max_leaf_nodes = 16
Accuracy =  88.37136929460581 %
Precision =  [88.37136929  0.          0.        ] %
Recall =  [100.   0.   0.] %
F1 score =  [93.82675257  0.          0.        ] %
[[17038     0     0]
 [ 1624     0     0]
 [  618     0     0]]


In [0]:
rnd_clf = RandomForestClassifier(n_estimators=1000, max_leaf_nodes=64, n_jobs=-1)
print("n_estimators = 1000, max_leaf_nodes = 64")
rnd_clf.fit(X_train_prepared, y_train)
y_pred = rnd_clf.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

n_estimators = 1000, max_leaf_nodes = 64
Accuracy =  88.37136929460581 %
Precision =  [88.37136929  0.          0.        ] %
Recall =  [100.   0.   0.] %
F1 score =  [93.82675257  0.          0.        ] %
[[17038     0     0]
 [ 1624     0     0]
 [  618     0     0]]


### 6.6. K Neighbors Classifier <a name="k_neighbors_classifier"></a>

In [0]:
neigh = KNeighborsClassifier(n_neighbors=3)
print("n_neighbors = 3")
neigh.fit(X_train_prepared, y_train)
y_pred = neigh.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

n_neighbors = 3
Accuracy =  85.97510373443984 %
Precision =  [88.88049228 18.20224719  9.38511327] %
Recall =  [96.64279845  4.98768473  4.69255663] %
F1 score =  [92.59925768  7.8298695   6.25674218] %
[[16466   335   237]
 [ 1500    81    43]
 [  560    29    29]]


In [0]:
neigh = KNeighborsClassifier(n_neighbors=5)
print("n_neighbors = 5")
neigh.fit(X_train_prepared, y_train)
y_pred = neigh.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

n_neighbors = 5
Accuracy =  87.44813278008299 %
Precision =  [88.67824893 21.05263158 16.52173913] %
Recall =  [98.5620378   2.95566502  3.07443366] %
F1 score =  [93.35927728  5.18358531  5.18417462] %
[[16793   164    81]
 [ 1561    48    15]
 [  583    16    19]]


In [0]:
neigh = KNeighborsClassifier(n_neighbors=10)
print("n_neighbors = 10")
neigh.fit(X_train_prepared, y_train)
y_pred = neigh.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

n_neighbors = 10
Accuracy =  88.21576763485477 %
Precision =  [88.53390316 22.78481013 21.42857143] %
Recall =  [99.70066909  1.10837438  0.48543689] %
F1 score =  [93.78605935  2.11391662  0.94936709] %
[[16987    45     6]
 [ 1601    18     5]
 [  599    16     3]]


In [0]:
neigh = KNeighborsClassifier(n_neighbors=20)
print("n_neighbors = 20")
neigh.fit(X_train_prepared, y_train)
y_pred = neigh.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

n_neighbors = 20
Accuracy =  88.3298755186722 %
Precision =  [88.44435207 25.          0.        ] %
Recall =  [99.90609226  0.49261084  0.        ] %
F1 score =  [93.82647999  0.96618357  0.        ] %
[[17022    14     2]
 [ 1616     8     0]
 [  608    10     0]]


In [0]:
neigh = KNeighborsClassifier(n_neighbors=30)
print("n_neighbors = 30")
neigh.fit(X_train_prepared, y_train)
y_pred = neigh.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

n_neighbors = 30
Accuracy =  88.350622406639 %
Precision =  [88.41181662 26.31578947  0.        ] %
Recall =  [99.9471769   0.30788177  0.        ] %
F1 score =  [93.82627621  0.60864273  0.        ] %
[[17029     9     0]
 [ 1619     5     0]
 [  613     5     0]]


In [0]:
neigh = KNeighborsClassifier(n_neighbors=40)
print("n_neighbors = 40")
neigh.fit(X_train_prepared, y_train)
y_pred = neigh.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

n_neighbors = 40
Accuracy =  88.350622406639 %
Precision =  [88.3986504  26.66666667  0.        ] %
Recall =  [99.95304613  0.24630542  0.        ] %
F1 score =  [93.82144726  0.4881025   0.        ] %
[[17030     8     0]
 [ 1620     4     0]
 [  615     3     0]]


In [0]:
neigh = KNeighborsClassifier(n_neighbors=50)
print("n_neighbors = 50")
neigh.fit(X_train_prepared, y_train)
y_pred = neigh.predict(X_test_prepared)
print("Accuracy = ", accuracy_score(y_test, y_pred) * 100, '%')
print("Precision = ", precision_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("Recall = ", recall_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print("F1 score = ", f1_score(y_test, y_pred, average=None, labels=['No Delay', 'Slight Delay', 'Heavy Delay']) * 100, '%')
print(confusion_matrix(y_test, y_pred, labels=['No Delay', 'Slight Delay', 'Heavy Delay']))

n_neighbors = 50
Accuracy =  88.3454356846473 %
Precision =  [88.39406208 21.42857143  0.        ] %
Recall =  [99.95304613  0.18472906  0.        ] %
F1 score =  [93.81886294  0.36630037  0.        ] %
[[17030     8     0]
 [ 1621     3     0]
 [  615     3     0]]
